In [2]:
import json
import string
import numpy as np

punc = string.punctuation

In [49]:
result_dic = {}
word_dic = load_word2vec(dim)
# set the dim
dim = 100
path = '/Users/zhengwei/Desktop/new_id.json'
# '/Users/zhengwei/Desktop/lesson6/社交网络挖掘/dataset/dblp-ref-0.json'
with open(path, 'r') as f:
    line = f.readline()
    temp_vec = np.zeros(dim)
    while line:        
        line = json.loads(line)
        if 'abstract' in line:
            abstract_ = line['abstract'].split(' ')
            length = len(abstract_)
            for item in abstract_:
                if item == '':
                    continue
                if item[0] in punc:
                    item = item[1:]
#                 if item == '':
#                     print(abstract_)
                if item == '':
                    continue
                if item[-1] in punc:
                    item = item[:-1]

                if item in word_dic:
                    temp_vec += np.array(word_dic[item], dtype=np.float64)
                else:
                    continue
            result_dic[line['id']] = [temp_vec/length]
        else:
            result_dic[line['id']] = [temp_vec]
        
        temp_vec = np.zeros(dim)
        title_ = line['title'].split(' ')
        length = len(title_)
        for item in title_:
            if item == '':
                    continue
            if item[0] in punc:
                item = item[1:]
            if item == '':
                    continue
            if item[-1] in punc:
                item = item[:-1]
                
            if item in word_dic:
                temp_vec += np.array(word_dic[item], dtype=np.float64)
            else:
                continue
        result_dic[line['id']].append(temp_vec/length)
        
        line = f.readline()

In [56]:
result_dic[1][0]

array([-4.77929687e-02,  2.45555391e-01,  2.34570080e-01,  4.96333547e-02,
        5.78041875e-02, -4.76344531e-02, -8.38733594e-02,  9.17767559e-02,
       -4.17209488e-01,  1.93618766e-01,  1.22915391e-01, -1.97329672e-01,
        1.84115539e-01,  5.61010312e-03,  1.75752755e-01, -4.62975156e-02,
        2.05226937e-01,  7.79395156e-02, -1.34265172e-01,  5.26235656e-02,
        1.13390703e-01, -2.12066266e-01,  2.77592884e-01,  1.89173203e-01,
        1.28171372e-01, -3.72696547e-01,  8.56345781e-02, -3.55042688e-01,
       -2.17762844e-01,  1.22383750e-02,  3.00649375e-02,  1.98880953e-01,
       -3.22155981e-01,  4.59024375e-02,  1.18302072e-01,  2.18529891e-01,
        2.04114625e-01,  2.75034750e-01, -1.77718109e-01, -1.61010670e-01,
       -3.31550528e-01, -3.57793734e-01, -6.22402500e-02, -8.19579625e-02,
       -8.00023141e-02,  6.63968281e-02,  1.84564328e-01, -3.40089984e-01,
       -2.12390291e-01, -5.96622594e-01,  2.87022969e-02,  2.47523453e-02,
        9.73137031e-02,  

In [41]:

def load_word2vec(dim):
    word_dic = {}
    with open('/Users/zhengwei/Desktop/glove/glove.6B.'+ str(dim) + 'd.txt') as f:
        line = f.readline()
        while line:
            line = line[:-1].split(' ')
            word_dic[line[0]] = line[1:]
            line = f.readline()
    return word_dic

In [57]:
import pickle

def write_in_pickle(data, path):
    output = open(path, 'wb')
    pickle.dump(data, output)
    output.close()

def load_pickle(path):
    pkl_file = open(path, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    return data

In [51]:
write_in_pickle(result_dic, '/Users/zhengwei/Desktop/vector.pkl')

In [1]:
### Task 2

import json

author_dic = {}
with open('/Users/zhengwei/Desktop/lesson6/社交网络挖掘/dataset/dblp-ref-0.json', 'r') as f:
    line = f.readline()
    while line:  
        line = json.loads(line)
        author_list = line['authors']
        for i in range(len(author_list)):
            main_author = author_list[i]
            if main_author not in author_dic:
                author_dic[main_author] = {}
            coo_author = author_list[:i] + author_list[i+1:]
            for author in coo_author:
                if author not in author_dic[main_author]:
                    author_dic[main_author][author] = 1
                else:
                    author_dic[main_author][author] += 1
        line = f.readline()

In [13]:
write_in_pickle(author_dic, '/Users/zhengwei/Desktop/coo_author.pkl')

In [3]:
data = load_pickle('/Users/zhengwei/Desktop/coo_author.pkl')

In [6]:
"""-------------------------Extract small and valuable dataset--------------------"""
import json
import string
import numpy as np
punc = string.punctuation

path = '/Users/zhengwei/Desktop/lesson6/社交网络挖掘/dataset/dblp-ref-all.json'
result_dic = {}
dim = 100
word_dic = load_word2vec(dim)

id_ref_num = {}
with open(path, 'r') as f:
    line = f.readline()
    while line:        
        line = json.loads(line)
        if 'references' in line and len(line['references']) > 0:
            for item in line['references']:
                if item not in id_ref_num:
                    id_ref_num[item] = 1
                else:
                    id_ref_num[item] += 1
        line = f.readline()

In [10]:
id_ref_list = sorted(id_ref_num.items(), key=lambda item: -item[1])
top_id = [item[0] for item in id_ref_list]
# map_oddID_newID = dict(zip(top_id, list(range(len(top_id)))))

In [29]:
# result_dic = {}
# dim = 100
# word_dic = load_word2vec(dim)
import time
file = open('/Users/zhengwei/Desktop/small_data.json', 'w') 
with open(path, 'r') as f:
    line = f.readline()
    i = 0
#    start =time.time()
    while line:    
        line = json.loads(line)
        if "abstract" in line and "references" in line:
            if "id" in line and line["id"] in top_id:
                i += 1
#                 if "references" in line and len(line['references']) > 0:
#                     new_refe = []
#                     for item in line['references']:
#                         if item in map_oddID_newID:
#                             new_refe.append(map_oddID_newID[item])
#                     line['references'] = new_refe
                file.write(json.dumps(line))
                file.write('\n')
            if i == 100000:
                break
        line = f.readline()
file.close()

In [21]:
len(top_id)

1985921

In [32]:

"""--------------Generate the new file with simplified ID----------------------"""
# result_dic = {}
# dim = 100
# word_dic = load_word2vec(dim)
import time
# file = open('/Users/zhengwei/Desktop/new_id.json', 'w') 
all_id = []
with open('/Users/zhengwei/Desktop/small_data.json', 'r') as f:
    line = f.readline()
    i = 0
#    start =time.time()
    while line:    
        line = json.loads(line)
        all_id.append(line["id"])
        line = f.readline()
#         if "abstract" in line and "references" in line:
#             if "id" in line and line["id"] in top_id:
#                 i += 1
# #                 if "references" in line and len(line['references']) > 0:
# #                     new_refe = []
# #                     for item in line['references']:
# #                         if item in map_oddID_newID:
# #                             new_refe.append(map_oddID_newID[item])
# #                     line['references'] = new_refe
#                 file.write(json.dumps(line))
#                 file.write('\n')
#             if i == 100000:
#                 break   
# file.close()
map_oddID_newID = dict(zip(all_id, list(range(len(all_id)))))

In [38]:
map_oddID_newID

{'001c58d3-26ad-46b3-ab3a-c1e557d16821': 0,
 '001c8744-73c4-4b04-9364-22d31a10dbf1': 1,
 '00bcf2d5-1592-46b0-81fd-933f90b5ecca': 2,
 '00c85316-bddf-4bcb-93f5-097adadd73c2': 3,
 '00dd5ece-1339-4cb6-a08e-43742ae2dac6': 4,
 '010d4ce9-0279-4166-ae73-14551ded6404': 5,
 '012b88ae-a763-45d6-8f19-2ec9ff739d5f': 6,
 '016a9a21-e882-4cda-a2b1-b2b46f10601e': 7,
 '01705f09-d395-4a0f-a89f-8d18382c81cc': 8,
 '01b6f2ca-3903-4196-a932-47a4be4c00bf': 9,
 '0265aea8-65f3-4f46-b8b1-6b65bcb1d38f': 10,
 '02a5e8a8-061e-4c1f-ab16-bda035578e54': 11,
 '034261a4-1e9b-463f-9549-c0b890af6a41': 12,
 '03990c4b-45be-4913-80d6-27d3b3381b36': 13,
 '03cda805-9746-48bb-a04d-02c2dac201c7': 14,
 '03dd4db1-3540-4de6-b8f4-5d79cc538dd7': 15,
 '04136c62-06a9-4c74-9da2-547448a9dc6f': 16,
 '05440797-8533-4496-b6ea-c414a4379fea': 17,
 '0572f1eb-2193-4dd6-b32e-b168806a1531': 18,
 '05e535a9-2021-4393-a19e-6a0277034323': 19,
 '074ff4d0-27b3-4854-8996-0c7adff14ce8': 20,
 '07de0f5d-2770-4950-affa-28f03dbc3c29': 21,
 '07f6d4ba-fb98-4988

In [48]:
file = open('/Users/zhengwei/Desktop/new_id.json', 'w') 
with open('/Users/zhengwei/Desktop/small_data.json', 'r') as f:
    line = f.readline()
    i = 0
    while line:    
        line = json.loads(line)
        line["id"] = map_oddID_newID[line["id"]]
        new_ref = []
        for item in line["references"]:
            if item in map_oddID_newID:
                new_ref.append(map_oddID_newID[item])
        line["references"] = new_ref
        file.write(json.dumps(line))
        file.write('\n')
        line = f.readline()
file.close()

In [58]:
"""-----------------TASK1-3-----------------"""
data = load_pickle('/Users/zhengwei/Desktop/cluster_KMeans.pkl')

In [66]:
class_data = dict()
for key, value in data.items():
    if value not in class_data:
        class_data[value] = [key]
    else:
        class_data[value].append(key)

dict_keys([3, 2, 0, 6, 5, 4, 8, 9, 1, 7])